In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('../../Downloads/assignment_data.xlsx')
dfID = df.drop_duplicates('ID')
dfID_NoID = dfID.drop('ID',axis=1)
dfID_NoIDR = dfID_NoID.reset_index().drop('index',axis=1)

In [3]:
count = 0
for i in dfID_NoIDR['Days Employed']:
    if float(i) > 0:
        count += 1
print(count)

967


In [4]:
CopyofdfID_NoIDR = dfID_NoIDR['Days Employed'].copy()
CopyofdfID_NoIDR[CopyofdfID_NoIDR > 0] = 0
dfID_NoIDR['Days Employed'] = CopyofdfID_NoIDR

In [11]:
dfID_NoIDR['Age'] = abs(dfID_NoIDR['Days Since Birth']/365.25)
dfID_NoIDR['Years in Employment'] = abs(dfID_NoIDR['Days Employed']/365.25)
dfNew = dfID_NoIDR.drop('Days Since Birth',axis=1)
dfYears = dfNew.drop('Days Employed',axis=1)

In [28]:
df_5000 = dfYears[0:5000]
df_5000

,Gender,Car,Real Estate,Children,Income,Payment Default,Age,Years in Employment
0,0,1,0,0,135000.0,1,28.032854,0.000000
1,0,0,1,0,202500.0,1,37.357974,14.713210
2,1,0,1,0,157500.0,1,45.524983,10.004107
3,0,0,0,1,270000.0,1,27.674196,4.032854
4,1,1,1,1,135000.0,0,42.384668,7.868583
...,...,...,...,...,...,...,...,...
4995,1,0,1,0,90000.0,1,50.817248,9.760438
4996,0,0,1,0,81000.0,1,27.033539,5.278576
4997,0,1,1,0,225000.0,0,56.643395,0.000000
4998,1,1,1,0,90000.0,1,30.554415,5.212868


In [29]:
import numpy as np

In [52]:
y = np.ravel(df_5000['Payment Default'])
y
#Your function is dumb

array([1, 1, 1, ..., 0, 1, 0])

In [65]:
Car = np.ravel(df_5000['Car'])
Gender = np.ravel(df_5000['Gender'])
RE = np.ravel(df_5000['Real Estate'])
Child = np.ravel(df_5000['Children'])
IN = np.ravel(df_5000['Income'])
AGE = np.ravel(df_5000['Age'])
YIE = np.ravel(df_5000['Years in Employment'])
X = np.array([Car,Gender,RE,Child,IN,AGE,YIE]).T

#X = df_5000[['Car','Gender','Real Estate','Children','Income','Age','Years in Employment']].loc[:5000].values

In [68]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0, stratify = y)

# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)
#print(y_train)
#print(y_test)

In [71]:
from sklearn.preprocessing import StandardScaler
np.set_printoptions(precision=3, suppress = True) # pretty printing

sc = StandardScaler()

sc.fit(X_train)

# print(dir(sc))
# print(sc.mean_, sc.scale_)


X_train_scaled = sc.transform(X_train)
print('means:', X_train.mean(axis=0), X_train_scaled.mean(axis=0))
print('sigmas', X_train.std(axis=0), X_train_scaled.std(axis=0))

X_test_scaled = sc.transform(X_test)
print('means:', X_test.mean(axis=0), X_test_scaled.mean(axis=0))
print('sigmas', X_test.std(axis=0), X_test_scaled.std(axis=0))

means: [     0.376      0.652      0.648      0.45  192123.022     43.558
      6.072] [ 0. -0. -0. -0. -0.  0. -0.]
sigmas [     0.484      0.476      0.478      0.807 115453.232     11.521
      6.486] [1. 1. 1. 1. 1. 1. 1.]
means: [     0.379      0.644      0.639      0.411 191404.599     43.699
      6.054] [ 0.007 -0.017 -0.019 -0.048 -0.006  0.012 -0.003]
sigmas [     0.485      0.479      0.48       0.71  102460.39      11.376
      6.576] [1.002 1.005 1.006 0.881 0.887 0.987 1.014]


In [72]:
from sklearn.linear_model import Perceptron

ppn = Perceptron(eta0=0.1, random_state=1)
ppn.fit(X_train_scaled, y_train)

print('--- Estimated Weights ---')
print('intercept:\n', ppn.intercept_)
print('coefficients:\n', ppn.coef_)

--- Estimated Weights ---
intercept:
 [-0.3]
coefficients:
 [[-0.18  -0.009 -0.221 -0.204  0.294 -0.213 -0.335]]


In [73]:
y_train_pred = ppn.predict(X_train_scaled)

# print('Predictions:', y_train_pred)
# print('True Labels:', y_train)

print('Number of incorrectly classified in training set:', (y_train != y_train_pred).sum())

error_train = (y_train != y_train_pred).sum()/len(y_train)

print(f'Misclassification Error: {error_train:.3f}')
print(f'Accuracy: {1 - error_train:.3f}')

print(40*'=')

y_test_pred = ppn.predict(X_test_scaled)
print('Number of incorrectly classified in test set:', (y_test != y_test_pred).sum())
error_test = (y_test != y_test_pred).sum()/len(y_test)

print(f'Misclassification Error: {error_test:.3f}')
print(f'Accuracy: {1 - error_test:.3f}')


Number of incorrectly classified in training set: 1685
Misclassification Error: 0.481
Accuracy: 0.519
Number of incorrectly classified in test set: 782
Misclassification Error: 0.521
Accuracy: 0.479


In [74]:
from sklearn.metrics import accuracy_score
print(f'Accuracy = {accuracy_score(y_test, y_test_pred):.3f}')

Accuracy = 0.479


In [75]:
print(f'Accuracy = {ppn.score(X_test_scaled, y_test):.3f}')

Accuracy = 0.479
